In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import os
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import homogeneity_score
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

In [2]:
df_metrics = pd.DataFrame(columns=['ARI_Louvain','ARI_kmeans','ARI_HC',
                                   'AMI_Louvain','AMI_kmeans','AMI_HC',
                                   'Homogeneity_Louvain','Homogeneity_kmeans','Homogeneity_HC'])

In [3]:
workdir = './output/'
path_fm = os.path.join(workdir,'feature_matrices/')
path_clusters = os.path.join(workdir,'clusters/')
path_metrics = os.path.join(workdir,'metrics/')
os.system('mkdir -p '+path_clusters)
os.system('mkdir -p '+path_metrics)

0

In [4]:
metadata = pd.read_csv('./input/metadata.tsv',sep='\t',index_col=0)
num_clusters = len(np.unique(metadata['label']))

In [5]:
files = [x for x in os.listdir(path_fm) if x.startswith('FM')]
len(files)

17

In [6]:
files

['FM_Control_BMnoisyp2.rds',
 'FM_BROCKMAN_BMnoisyp2.rds',
 'FM_Cusanovich2018_BMnoisyp2.rds',
 'FM_cisTopic_BMnoisyp2.rds',
 'FM_chromVAR_BMnoisyp2_kmers.rds',
 'FM_chromVAR_BMnoisyp2_motifs.rds',
 'FM_chromVAR_BMnoisyp2_kmers_pca.rds',
 'FM_chromVAR_BMnoisyp2_motifs_pca.rds',
 'FM_GeneScoring_BMnoisyp2.rds',
 'FM_GeneScoring_BMnoisyp2_pca.rds',
 'FM_Cicero_BMnoisyp2.rds',
 'FM_Cicero_BMnoisyp2_pca.rds',
 'FM_SnapATAC_BMnoisyp2.rds',
 'FM_Scasat_BMnoisyp2.rds',
 'FM_scABC_BMnoisyp2.rds',
 'FM_SCRAT_BMnoisyp2.rds',
 'FM_SCRAT_BMnoisyp2_pca.rds']

In [7]:
def getNClusters(adata,n_cluster,range_min=0,range_max=3,max_steps=20):
    this_step = 0
    this_min = float(range_min)
    this_max = float(range_max)
    while this_step < max_steps:
        print('step ' + str(this_step))
        this_resolution = this_min + ((this_max-this_min)/2)
        sc.tl.louvain(adata,resolution=this_resolution)
        this_clusters = adata.obs['louvain'].nunique()
        
        print('got ' + str(this_clusters) + ' at resolution ' + str(this_resolution))
        
        if this_clusters > n_cluster:
            this_max = this_resolution
        elif this_clusters < n_cluster:
            this_min = this_resolution
        else:
            return(this_resolution, adata)
        this_step += 1
    
    print('Cannot find the number of clusters')
    print('Clustering solution from last iteration is used:' + str(this_clusters) + ' at resolution ' + str(this_resolution))

In [8]:
for file in files:
    file_split = file.split('_')
    method = file_split[1]
    dataset = file_split[2].split('.')[0]
    if(len(file_split)>3):
        method = method + '_' + '_'.join(file_split[3:]).split('.')[0]
    print(method)

    pandas2ri.activate()
    readRDS = robjects.r['readRDS']
    df_rds = readRDS(os.path.join(path_fm,file))
    fm_mat = pandas2ri.ri2py(robjects.r['data.frame'](robjects.r['as.matrix'](df_rds)))
    fm_mat.columns = metadata.index
    
    adata = sc.AnnData(fm_mat.T)
    adata.var_names_make_unique()
    adata.obs = metadata.loc[adata.obs.index,]
    df_metrics.loc[method,] = ""
    #Louvain
    sc.pp.neighbors(adata, n_neighbors=15,use_rep='X')
#     sc.tl.louvain(adata)
    getNClusters(adata,n_cluster=num_clusters)
    #kmeans
    kmeans = KMeans(n_clusters=num_clusters, random_state=2019).fit(adata.X)
    adata.obs['kmeans'] = pd.Series(kmeans.labels_,index=adata.obs.index).astype('category')
    #hierachical clustering
    hc = AgglomerativeClustering(n_clusters=num_clusters).fit(adata.X)
    adata.obs['hc'] = pd.Series(hc.labels_,index=adata.obs.index).astype('category')
    #clustering metrics
    
    #adjusted rank index
    ari_louvain = adjusted_rand_score(adata.obs['label'], adata.obs['louvain'])
    ari_kmeans = adjusted_rand_score(adata.obs['label'], adata.obs['kmeans'])
    ari_hc = adjusted_rand_score(adata.obs['label'], adata.obs['hc'])
    #adjusted mutual information
    ami_louvain = adjusted_mutual_info_score(adata.obs['label'], adata.obs['louvain'],average_method='arithmetic')
    ami_kmeans = adjusted_mutual_info_score(adata.obs['label'], adata.obs['kmeans'],average_method='arithmetic')   
    ami_hc = adjusted_mutual_info_score(adata.obs['label'], adata.obs['hc'],average_method='arithmetic')
    #homogeneity
    homo_louvain = homogeneity_score(adata.obs['label'], adata.obs['louvain'])
    homo_kmeans = homogeneity_score(adata.obs['label'], adata.obs['kmeans'])
    homo_hc = homogeneity_score(adata.obs['label'], adata.obs['hc'])

    df_metrics.loc[method,['ARI_Louvain','ARI_kmeans','ARI_HC']] = [ari_louvain,ari_kmeans,ari_hc]
    df_metrics.loc[method,['AMI_Louvain','AMI_kmeans','AMI_HC']] = [ami_louvain,ami_kmeans,ami_hc]
    df_metrics.loc[method,['Homogeneity_Louvain','Homogeneity_kmeans','Homogeneity_HC']] = [homo_louvain,homo_kmeans,homo_hc] 
    adata.obs[['louvain','kmeans','hc']].to_csv(os.path.join(path_clusters ,method + '_clusters.tsv'),sep='\t')

Control


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 6 at resolution 1.5
BROCKMAN


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 10 at resolution 1.5
step 1
got 6 at resolution 0.75
Cusanovich2018


...

step 0
got 14 at resolution 1.5
step 1
got 3 at resolution 0.75
step 2
got 4 at resolution 1.125
step 3
got 9 at resolution 1.3125
step 4
got 7 at resolution 1.21875
step 5
got 5 at resolution 1.171875
step 6
got 7 at resolution 1.1953125
step 7
got 5 at resolution 1.18359375
step 8
got 6 at resolution 1.189453125
SCRAT


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 8 at resolution 1.5
step 1
got 6 at resolution 0.75
SCRAT_pca


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 11 at resolution 1.5
step 1
got 6 at resolution 0.75


In [9]:
df_metrics.to_csv(path_metrics+'clustering_scores.csv')

In [10]:
df_metrics

,ARI_Louvain,ARI_kmeans,ARI_HC,AMI_Louvain,AMI_kmeans,AMI_HC,Homogeneity_Louvain,Homogeneity_kmeans,Homogeneity_HC
Control,0.965315,0.798218,0.797901,0.965279,0.868754,0.865926,0.96539,0.868205,0.866151
BROCKMAN,0.94724,0.657768,0.725251,0.949048,0.75279,0.804234,0.949222,0.752645,0.800707
Cusanovich2018,1,0.773251,0.997997,1,0.900015,0.997053,1,0.871049,0.99707
cisTopic,1,0.997997,0.997997,1,0.997053,0.997053,1,0.99707,0.99707
chromVAR_kmers,0.76863,0.71703,0.649466,0.82417,0.78671,0.730611,0.82211,0.787383,0.726742
chromVAR_motifs,0.464693,0.459593,0.421048,0.612146,0.614561,0.579887,0.610832,0.6164,0.579137
chromVAR_kmers_pca,0.741652,0.750573,0.702112,0.803962,0.809526,0.768438,0.804208,0.810403,0.765538
chromVAR_motifs_pca,0.483201,0.451405,0.432268,0.624937,0.606774,0.601031,0.621348,0.608964,0.588254
GeneScoring,0.0214239,0.448314,0.362124,0.0263894,0.601596,0.447831,0.0340632,0.521397,0.409118
GeneScoring_pca,0.401574,0.403772,0.400471,0.499477,0.492295,0.498276,0.490116,0.481476,0.484532
